In [2]:
q = """
CREATE TABLE IF NOT EXISTS ml_tmp_db.mix_sld_ds_test (
    t varchar,
    black_number integer,
    num_perteam integer,
    cnt integer,
    start_time varchar,
    end_time varchar
);

INSERT INTO ml_tmp_db.mix_sld_ds_test

WITH matching AS (
    SELECT *
    FROM "10060_oss_bi_dw"."solarland_match_end"
    WHERE dt = '{dt}' 
        AND mode_id in (5,7)
        -- AND event_time BETWEEN '{start_time}' AND '{end_time}'
)

, starting as (
    SELECT *
    FROM "10060_oss_bi_dw"."solarland_match_team_start"
    WHERE dt = '{dt}' 
        -- AND event_time BETWEEN '{start_time}' AND '{end_time}'
)

, tmp AS (
    SELECT 
        min(s.event_time) as start_time,
        min(m.event_time) as end_time,
        max(black_number) as black_number,
        max(num_perteam) as num_perteam,
        pool_id
    FROM matching AS m
    LEFT JOIN starting AS s
    ON m.ticket_id = s.ticket_id
        AND m.match_server_id = s.match_server_id
    INNER JOIN (  
        select acc_id, acc_create_country 
        from "10060_bi_ods"."total_account_attr_all"
        WHERE acc_create_country = 'PH'
    ) AS acc
    ON acc.acc_id = m.open_id
    GROUP BY m.ticket_id, pool_id
)


select 
	t.t, tmp.black_number, tmp.num_perteam, count(1) AS cnt, '{start_time}' AS start_time, '{end_time}' AS end_time
from tmp
CROSS JOIN UNNEST(
  TRANSFORM(SEQUENCE(0, 60 * 24 * 60 / {interval}), x-> DATE_FORMAT(DATE_ADD('second', x * {interval}, '{start_time}'), '%Y-%m-%d %H:%i:%s'))
) AS t(t)
WHERE tmp.start_time <= t.t AND tmp.end_time >= t.t
GROUP BY t.t, tmp.black_number, tmp.num_perteam 

"""
import os;os.environ['HIVESERVER2_HOST'] = 'bbx.farlightgames.com'
import sys;sys.path.append('../../')
from config.HIVESERVER2_token import *
from pyutil.pysql.sql import SQLClient

# conf = dict(interval = 20, start_time = '2023-10-22 19:58:00', end_time = '2023-10-22 22:02:00', dt = '2023-10-22')

# client = SQLClient('mix')
# [client.execute_trino_query(_q.format(**conf)) for _q in q.split(';')]

# conf = dict(interval = 20, start_time = '2023-10-22 00:00:00', end_time = '2023-10-22 14:02:00', dt = '2023-10-22')

# client = SQLClient('mix')
# [client.execute_trino_query(_q.format(**conf)) for _q in q.split(';')]

In [3]:
# client = SQLClient('mix')
# client.execute_trino_query('select * from ml_tmp_db.mix_sld_ds_test2')
# ds = client.to_pandas()


# import plotly.express as px
# from plotly.subplots import make_subplots

# ds0 = ds.query("num_perteam == 4")
# ds1 = ds0.assign(cnt = lambda x: x.cnt * x.black_number).groupby('t').cnt.sum().reset_index()

# fig0 = px.area(
#     ds0,
#     x = 't', 
#     y = 'cnt', 
#     color = "black_number", 
#     line_group="black_number",
#     category_orders={"black_number" : [4,3,2,1]},
#     title = '匹配队列队伍数量'.format(**conf)
# )
# fig0.update_layout(legend = dict(title='大厅组队人数'))
# fig1 = px.line(ds1, x = 't', y = 'cnt', title = '匹配队列总人数')

# fig0.show()
# fig1.show()

# # fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=['匹配队列队伍数量', '匹配队列总人数'], row_heights=[0.7, 0.3])
# # fig.add_traces(fig0.data,1,1)
# # fig.add_traces(fig1.data,2,1)


In [2]:
q = '''
WITH matching AS (
    SELECT *
    FROM "10060_oss_bi_dw"."solarland_match_end"
    WHERE dt = '2023-10-22' 
        AND mode_id in (5,7)
        AND match_server_id = 'matcher-6867686467-fdtmr'
        AND event_time BETWEEN '2023-10-22 12:00:00'  AND '2023-10-22 12:30:00'
)

, starting as (
    SELECT *
    FROM "10060_oss_bi_dw"."solarland_match_team_start"
    WHERE dt = '2023-10-22' 
        AND match_server_id = 'matcher-6867686467-fdtmr'
        AND event_time BETWEEN '2023-10-22 12:00:00'  AND '2023-10-22 12:30:00'
)

SELECT 
    s.ticket_id, num_perteam, pool_id, black_number, MIN(s.event_time) as start_time, MIN(m.event_time) as end_time
FROM matching AS m
INNER JOIN starting AS s
ON m.ticket_id = s.ticket_id
    AND m.match_server_id = s.match_server_id
GROUP BY s.ticket_id, num_perteam, pool_id, black_number
'''

import os;os.environ['HIVESERVER2_HOST'] = 'bbx.farlightgames.com'
import sys;sys.path.append('../../')
from config.HIVESERVER2_token import *
from pyutil.pysql.sql import SQLClient

client = SQLClient('mix')
client.execute_trino_query(q)
ds = client.to_pandas()


In [3]:
ds.to_csv('../../data/raw/matchmaking.1024.csv', index = False)

In [7]:
a = {1:2,3:4}


In [27]:
ds.groupby('start_time').ticket_id.count().sort_index().to_csv('test.csv')

In [7]:
import tensorflow as tf

batch_size = 4
items_w = tf.tile(tf.constant([[1, 0, 0, 0, 0, 0, 0]], dtype=tf.float32), [batch_size, 1])
# items_w = tf.reshape(items_w, (batch_size, -1))

In [10]:
items = tf.tile(tf.constant(['vip_point'] * 7, dtype=tf.string), [batch_size])
items = tf.reshape(items, (batch_size, -1))

items

<tf.Tensor: shape=(4, 7), dtype=string, numpy=
array([[b'vip_point', b'vip_point', b'vip_point', b'vip_point',
        b'vip_point', b'vip_point', b'vip_point'],
       [b'vip_point', b'vip_point', b'vip_point', b'vip_point',
        b'vip_point', b'vip_point', b'vip_point'],
       [b'vip_point', b'vip_point', b'vip_point', b'vip_point',
        b'vip_point', b'vip_point', b'vip_point'],
       [b'vip_point', b'vip_point', b'vip_point', b'vip_point',
        b'vip_point', b'vip_point', b'vip_point']], dtype=object)>

In [ ]:
tf.